# How often is 'x' mentioned on Twitter?
##### *Counts pulled with the Twarc library. [Check it out](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/)*. 

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

#### Read data

In [4]:
# mentioned = "Elon Musk"

# src = pd.read_csv(
#     "../../data/raw/twitter_mentions/elonmusk_mentions_daily_full_name.csv",
#     parse_dates=["start", "end"],
# ).sort_values("start", ascending=False)

#### First five rows

#### Process dates

In [5]:
# src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
# src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
# src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

#### Slim down and re-order the dataframe

#### When was the first mention? 

#### Define that as a variable

#### How many total mentions? 

#### Average mentions? 

#### Make a new dataframe starting from first mention

#### Which day was mentioned the most? 

#### Chart it

In [6]:
# alt.Chart(source).mark_area().encode(
#     x="year:T",
#     y="net_generation:Q",
# )

---

## Aggregate 

#### Groupby month/year

#### Which month-year was max? 

---

#### Exports

In [7]:
# df_all_months.to_csv(f"../../students/YOUR_FOLDER/data/twitter_mentions_{mentioned}.csv", index=False)